This notebook will create the dataset(s) necessary to hunt our most Bayesian game.
This will require:
1) A dataset, along with counts (for probability calculation) of the order of events for every bill prior to 2018, labeled t1 through tn
2) A dataset that represents the additional feature data for each bill at each stage (docVec cluster, sponsor, etc).
    a) If possible, comb news sources from around that time for a sentiment analysis
    b) Maybe, for fun, get data from the major NH newspapers with editorials and the like?
3) Merge the two data sets

This data set will be used to calculate the Bayesian probability at the DocCluster level.

See Data_Dictionary.pdf for the data structure of for our data.
    

In [5]:
# This cell will accumulate a list of urls from legiscan that will be used for the text extraction
#from google.cloud import storage
import json
import pickle
import string
import pandas as pd
import dateutil
from datetime import date
import re
 
#load the bill clusters from previous notebooks------------------------------------------------
with open('NHlegdata.dict','rb') as f:
    leg_data = pickle.load(f)
    
with open('committee_types.dict', 'rb') as f:
    committee_type = pickle.load(f)
print(committee_type['S'][1])

Capital Budget Committee


In [2]:
def remove(value, deletechars):
    for c in deletechars:
        value = value.replace(c,'')
    return value

#reduces the event description on a bill into the key words and phrases
#defined in the classifier words list
def simplify_bill_event(bill_event, classifier_keywords):
    temp_stat = []
    for k in nh_status_keywords:
        pos = s.find(k)
        if pos != -1:
            temp_stat.append([k,pos])
    temp_stat.sort(key=lambda x: x[1])
    simplified_event = ' '.join(item[0] for item in temp_stat)
    return simplified_event


-----------Only Needs to be run the first time, data loaded into a binary file------------------------------

In [3]:
# create storage client
storage_client = storage.Client.from_service_account_json('cedc-erc-legislation-project-10835c838715.json')

# get bucket with name
bucket = storage_client.get_bucket('cedc-erc-storage')

# get bucket data as blob
#max results restricted to 5 for testing
bloblist = bucket.list_blobs(prefix='LegData/NH')

In [ ]:
# THIS CREATES NH_bill_dict


# create storage client
storage_client = storage.Client.from_service_account_json('cedc-erc-legislation-project-10835c838715.json')

# get bucket with name
bucket = storage_client.get_bucket('cedc-erc-storage')

# get bucket data as blob
#max results restricted to 5 for testing
bloblist = bucket.list_blobs(prefix='LegData/NH')

bill_dict = {}
parse_bill = '{"bill":'
regex_year = r"(\d+)"


for bl in bloblist:
    if '.json' in bl.name and '/bill/' in bl.name:
        temp = bl.download_as_string().decode("utf-8")
        _temp = json.loads(re.sub(parse_bill, "", temp[:-1]))
        tempy = bl.name
        match = re.search(regex_year, tempy) 
        year = int(match.group(0))
        if not year in bill_dict:
            bill_dict[year] = []
        else:
            bill_dict[year].append(_temp)

#drop to a binary file so the above doesn't need to be run each time
with open('NH_bill_dict','wb') as f:
    pickle.dump(bill_dict, f)

In [134]:
# THIS CREATES NH_vote_dict


# create storage client
storage_client = storage.Client.from_service_account_json('cedc-erc-legislation-project-10835c838715.json')

# get bucket with name
bucket = storage_client.get_bucket('cedc-erc-storage')

# get bucket data as blob
#max results restricted to 5 for testing
bloblist = bucket.list_blobs(prefix='LegData/NH')

vote_dict = {}
parse_vote = '{"roll_call":'
regex_year = r"(\d+)"

for bl in bloblist:
    if '.json' in bl.name and '/vote/' in bl.name:
        temp = bl.download_as_string().decode("utf-8")
        _temp = json.loads(re.sub(parse_bill, "", temp[:-1]))
        tempy = bl.name
        match = re.search(regex_year, tempy) 
        year = int(match.group(0))
        if not year in vote_dict:
            vote_dict[year] = []
        else:
            vote_dict[year].append(_temp)
        
with open('NH_vote_dict','wb') as f:
    pickle.dump(vote_dict, f)

2016
2017
2018
2019
2015


In [136]:
# THIS CREATES NH_people_dict

# create storage client
storage_client = storage.Client.from_service_account_json('cedc-erc-legislation-project-10835c838715.json')

# get bucket with name
bucket = storage_client.get_bucket('cedc-erc-storage')

# get bucket data as blob
#max results restricted to 5 for testing
bloblist = bucket.list_blobs(prefix='LegData/NH')


people_dict = {}
parse_people = '{"person":'
regex_year = r"(\d+)"


for bl in bloblist:
    if '.json' in bl.name and '/people/' in bl.name:
        temp = bl.download_as_string().decode("utf-8")
        _temp = json.loads(re.sub(parse_people, "", temp[:-1]))
        tempy = bl.name
        match = re.search(regex_year, tempy) 
        year = int(match.group(0))
        if not year in people_dict:
            people_dict[year] = []
        else:
            people_dict[year].append(_temp)
        
with open('NH_people_dict','wb') as f:
    pickle.dump(people_dict, f)
    


2016
2017
2018
2019
2015


In [ ]:
#Create the python dictionary for all of the bills we're looking at
#json_data = {'bills':}
json_raw = ''
#json_raw = '{'
#json_raw = '{\"bills\":['
for bl in bloblist:
    if '.json' in bl.name and '/bill/' in bl.name and '2015' not in bl.name and '2016' not in bl.name:
        json_raw += bl.download_as_string().decode("utf-8") + ','


#remove the last comma and close out the list and root
json_raw = json_raw[:-1] + '}'


json_data = json.loads(new_json_raw)

#drop to a binary file so the above doesn't need to be run each time
#with open('NHlegdata.dict','wb') as f:
#    pickle.dump(json_data, f)

-----------------------------------------------------------------------------------------------------------------

In [3]:
with open('NHlegdata.dict','rb') as f:
    leg_data = pickle.load(f)

with open('NHlegclusters.dict','rb') as f:
    leg_clusters = pickle.load(f)

In [4]:
#keywords we're looking for in bill events that allow us to simplify the event descriptions
#note: 'introduced and adopted' was not in the list but frequent in passing bills so i added it to the end
nh_status_keywords = ['amendment adopted','bill killed','children and family law','commerce'
,'committee','committee amendment','report','consumer affairs','criminal justice'
,'died on table','education','election law','energy','environment and agriculture'
,'executive departments and administration','executive session','finance','fish and game'
,'hearing','inexpedient to legislate','interim study'
,'internal affairs','introduced','judiciary','labor, industrial and rehabilitative services'
,'lay on table','legislative administration','majority committee report'
,'minority committee report','motion adopted','motion failed'
,'moved inexpedient to legislate','municipal and county government','natural resources'
,'ought to pass','ought to pass with amendment','public'
,'public works and highways','referred to','removed from consent'
,'resources, recreation and development','retained in committee','science, technology'
,'state-federal relations','to be introduced','transportation','vacated to','voice'
,'vote','ways and means','without objection','elderly affairs','workforce development'
,'municipal affairs','health','human services','veterans affairs','marine resources'
,'public safety','vacated and referred to','vacated from','enrolled','signed', 'special order'
,'subcommittee', 'work session','retained bill','cancelled','rescheduled','next session day'
,'beginning of regular calendar','law without signature', 'continued','recessed','vetoed'
,'veto sustained','floor amendment','house non-concurs with senate','senate non-concurs with house'
,'speaker appoints','speaker appoints alternates','request for committee of conference','accedes to'
,'requests cofc','request cofc','request committee of conference','requests committee of conference', 'introduced and adopted']

In [5]:
#print ordered list of key words
#for x in sorted(nh_status_keywords):
    #print (x)

In [6]:
# passing keywords list
nh_passing_keywords =['ought to pass' ,'ought to pass with amendment','inexpedient to legislate'
                     ,'signed', 'law without signature', 'motion adopted', 'introducted and adopted']
for x in sorted(nh_passing_keywords):
    print (x)

inexpedient to legislate
introducted and adopted
law without signature
motion adopted
ought to pass
ougut to pass with amendment
signed


In [7]:
#define the lists that will make up the dictionary loaded into pandas---------------
bill_id_list = []
time_t_list = []
hist_list = [[]]
next_hist_list = [[]]
event_list = []
next_event_list=[]
sponsor_list = []
sponsor_vec_list = []
sponsor_vec2=[]
bill_category_list =[]
event_out_desc_list = []
event_out_metric_list = []
doc_id_list = []
year_list = []
session_list = []
chamber_list = []
win_state_list=[]
overall_passed_list=[]
public_sentiment=[]
public_knowledge=[]
#------------------------------------------------------------------------------------
#Grab the lists
for b in leg_data['bills']:
    temp_hist_list=[]
    temp_next_hist_list = []
    bill_temp = b['bill']['bill_id']
    year_temp = b['bill']['session']['year_end']
    session_id_temp = b['bill']['session']['session_id']
    #as time, t, is based on the events that occur on a bill, the lists will include duplicates, as necessary, that align to the number of t instances
    for i,h in enumerate(b['bill']['history'],start=1):
        temp_hist_list.append(remove(str(h['action']).lower(),'\/:*?"<>|0123456789.'))
        chamber_list.append(str(h['chamber']))
        time_t_list.append(i) 
        bill_id_list.append(bill_temp)
        year_list.append(year_temp)
        session_list.append(session_id_temp)
        sponsor_vector = 0.0000
        total_sponsors = 0.0
        #grab the next event, which is our label for the MN_NB
        try:
            next_event = b['bill']['history'][i]
        except:
            next_event = h
            
        temp_next_hist_list.append(remove(str(next_event['action']).lower(),'\/:*?"<>|0123456789.'))
        
        for x in b['bill']['sponsors']:
            if x['party'] == 'D':
                sponsor_vector+=-1
            elif x['party'] == 'R':
                sponsor_vector+=1
            total_sponsors += 1
        if total_sponsors == 0:
            sponsor_list.append(0)
        else:
            sponsor_list.append(sponsor_vector/total_sponsors)   
    #---------------------------------------------------------------------------------------------------------    
        #grab the aossciated doc_ids to pull in the cluster information
        #use the date of the bill revisions to determine
    #---------------------------------------------------------------------------------------------------------
        if len(b['bill']['texts']) == 0:
            doc_id_list.append(0)
            
        else:
            for j,d in enumerate(b['bill']['texts']):

                doc_date_str = d['date']

                #determine the next date in the list to create a date range to
                #check for
                try:
                    next_doc = b['bill']['texts'][j+1]
                    doc_next_date = dateutil.parser.isoparse((next_doc['date'])).date()

                except:
                    doc_next_date = date(2100,1,1)

                hist_date_string = h['date']
                hist_date = dateutil.parser.isoparse(hist_date_string).date()

                if doc_date_str == '0000-00-00':
                    doc_date = date(2000,1,1)
                else:
                    doc_date = dateutil.parser.isoparse(doc_date_str).date()

                if hist_date >= doc_date and hist_date < doc_next_date:
                    doc_id_list.append(d['doc_id'])
    #---------------------------------------------------------------------------------------------------------
    hist_list.append(temp_hist_list)
    next_hist_list.append(temp_next_hist_list)
    

In [8]:
#process the hist_list to create the simiplified event list (use keywords to simplify event descriptions)
#uses the UDF
for h in hist_list:
    for s in h:
        event_list.append(simplify_bill_event(s,nh_status_keywords))
for h in next_hist_list:
    for s in h:
        next_event_list.append(simplify_bill_event(s,nh_status_keywords))

In [9]:
#Get the cluster assignment for the document ids
for d in doc_id_list:
    try:
        ci = leg_clusters['doc_id'].index(str(d))
        bill_category_list.append(leg_clusters['cluster'][ci])
    except:
        bill_category_list.append('X')
        #print(leg_clusters['doc_id'])

In [10]:
print(len(bill_id_list),'\n'
            ,len(time_t_list),'\n'
            ,len(bill_category_list),'\n'
            ,len(event_list),'\n'
            ,len(sponsor_list),'\n'
            ,len(next_event_list),'\n'
            ,len(session_list),'\n'
            ,len(year_list),'\n'
            ,len(chamber_list))

34014 
 34014 
 34014 
 34014 
 34014 
 34014 
 34014 
 34014 
 34014


In [19]:
#Create a pandas dataframe from the lists
leg_dict = {'bill_id':bill_id_list
            ,'time_t':time_t_list
            ,'bill_cat':bill_category_list
            ,'event':event_list
            ,'sponsor_vec':sponsor_list
            ,'next_event':next_event_list
            ,'session':session_list
            , 'year':year_list
            , 'chamber':chamber_list}


leg_dict2= {'bill_id':bill_id_list
            ,'time_t':time_t_list
            ,'bill_cat':bill_category_list
            ,'event':event_list
            ,'sponsor_vec2':sponsor_vec2
            ,'session':session_list
            , 'year':year_list
            , 'public knowledge': public_knowledge
            , 'public sentiment': public_sentiment
            , 'chamber':chamber_list}

leg_dict3={#'last stage passed':win_state_list
            'every stage passed':overall_passed_list}


leg_data_dict = {'time_t':time_t_list
            ,'bill_cat':bill_category_list
            ,'event':event_list
            ,'sponsors':sponsor_list}

leg_labels = next_event_list
leg_labels2= overall_passed_list
leg_DF = pd.DataFrame(leg_dict)
#leg_DF.set_index(['bill_id','time_t'],inplace=True)

In [12]:
#Define the win state
win_state_list=[]
x=0
for b in leg_dict['event']:
    if b in nh_passing_keywords:
        x+=1
        win_state_list.append(1)
    else:   
        win_state_list.append(0)
print(x)

873


In [13]:
#Define a variable which has 1 at any time stage to use for overall model
temp_var=[0]*len(bill_id_list)
current_bill = 1
cur_id = bill_id_list[0]

for y in range(0, len(bill_id_list)): 
    if cur_id == bill_id_list[y]:
        temp_var[y] = current_bill
    else:
        current_bill = current_bill + 1   
        temp_var[y] = current_bill      
    cur_id = bill_id_list[y] 
print(temp_var[len(temp_var)-2])

2979


In [14]:
overall_passed_list = [0]*len(bill_id_list)

for x in range(len(temp_var)-2,-1,-1):   
    if win_state_list[x] == 1:
        overall_passed_list[x] = 1
        
    elif overall_passed_list[x+1] == 1 and temp_var[x] == temp_var[x+1]:
        overall_passed_list[x] = 1
        
    else:
        overall_passed_list[x] = 0

In [15]:
j=0
for x in range(len(win_state_list)):
    if win_state_list[x]==1:
        j+=1
print(j)
total_bills_passed=j
total_bills= 2979
print(j/2979*100)

873
29.305135951661633


In [16]:
# Public Sentiment Data
public_sentiment=[0]*len(bill_id_list)
for i in range(len(bill_id_list)):
    public_sentiment[i]=0.5

In [17]:
# Public Knoweldge Data
public_knowledge=[0]*len(bill_id_list)
for i in range(len(bill_id_list)):
    public_knowledge[i]=0.5

In [18]:
#Adjusting the sponsor vec to not have negative values for the Bayes model
sponsor_vec2=[0]*len(sponsor_list)
for i in range(0, len(sponsor_list)-1):
    sponsor_vec2[i]=0.5*sponsor_list[i]+0.5
#print(sponsor_list[0:40],sponsor_vec2[0:40])  

In [20]:
#creating the overall dataset with win state, and overall_passed which has a value of 1 for all doc ids or in other words time states of each bill
overall_leg_DF = pd.DataFrame(leg_dict2)

# creating an overall dataset with defined win condition
with open('NHlegoverall.dict','wb') as f:
    pickle.dump(leg_dict2, f)
    
with open('NHlegoverall.DF','wb') as f:
    pickle.dump(overall_leg_DF, f)
    
overall_leg_DF.to_csv('overall_leg_data2.csv')

In [21]:
#creating the target dataset for overall classifiers leg_win_state.csv
win_state_DF = pd.DataFrame(leg_dict3)

# creating an overall dataset with defined win condition
with open('NHWinState.dict','wb') as f:
    pickle.dump(leg_dict3, f)
    
with open('NHWinState.DF','wb') as f:
    pickle.dump(win_state_DF, f)
    
win_state_DF.to_csv('leg_win_state.csv')

In [22]:
#labels 2 as the 1 passed 0 failed but at every stage
with open('NHlegoverallLabel.list','wb') as f:
    pickle.dump(leg_labels2, f)

In [23]:
#leg_data.csv
with open('NHLegTimeSeriesDS.DF','wb') as f:
    pickle.dump(leg_DF, f)

with open('NHLegTimeSeriesDS.dict','wb') as f:
    pickle.dump(leg_dict, f)

with open('NHLegTimeSeriesData.dict','wb') as f:
    pickle.dump(leg_data_dict, f)

with open('NHLegTimeSeriesLabels.list','wb') as f:
    pickle.dump(leg_labels, f)

In [24]:
print(len(next_event_list),len(event_list))
print(len(win_state_list)) 

34014 34014
34014


In [25]:
print(leg_labels[0:10])

['public hearing', 'executive session', 'committee report inexpedient to legislate vote', 'inexpedient to legislate motion adopted voice vote', 'inexpedient to legislate motion adopted voice vote', 'vacated to children and family law without objection', 'public hearing', 'executive session', 'committee report inexpedient to legislate vote', 'removed from consent']


In [26]:
#print(hist_list[1743:1748])
#print(bill_id_list[:20])
#print(time_t_list[:20])
#print(bill_category_list[:20])

In [27]:
leg_DF.to_csv('leg_data.csv')

In [28]:
for b in leg_data['bills']:
    if b['bill']['bill_id'] == 1135962:
        print(b['bill']['bill_number'],  b['bill']['session']['year_end'], b['bill']['texts'][0]['doc_id'], b['bill']['texts'][1]['doc_id'])

HB105 2019 1833342 2030127
